<a href="https://colab.research.google.com/github/S0HYUNN/Style_Focus/blob/DB_data/%EA%B2%8C%EC%8B%9C%EB%AC%BC_%EC%88%98%EC%A7%91_%EB%A8%B8%EC%8B%A0_style_focus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 수집 및 csv파일 변화-저장

import requests
import pandas as pd

def get_facebook_data(url):
    response = requests.get(url)
    data = response.json()
    return data

def extract_user_info(data):
    user_info = {
        'username': data.get('username', ''),
        'follows_count': data.get('follows_count', 0),
        'profile_picture_url': data.get('profile_picture_url', ''),
        'ig_id': data.get('ig_id', ''),
        'followers_count': data.get('followers_count', 0),
        'media_count': data.get('media_count', 0),
    }
    return user_info

def extract_media_info(data):
    media_list = []
    media_data = data.get('media', {}).get('data', [])
    for media in media_data:
        media_info = {
            'caption': media.get('caption', ''),
            'comments_count': media.get('comments_count', 0),
            'media_id': media.get('id', ''),
            'like_count': media.get('like_count', 0),
            'media_product_type': media.get('media_product_type', ''),
            'media_type': media.get('media_type', ''),
            'media_url': media.get('media_url', ''),
            'timestamp': media.get('timestamp', ''),
            'username': media.get('username', ''),
        }
        media_list.append(media_info)

    return media_list

def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)


if __name__ == "__main__":
    user_name = input()
    business_ig_id = "17841405482908356"
    access_token = "EAADyzhuZC910BO6LCJhJGZB6UUMZBBGZAbqoleoi6oHaz6jF3mNBoEs7a1ZCpcxo7Od7OaNv8G6hFdg2hZCoqOZATzEvxxrBl28jA9fxgDbujSl0kDi96EGzDpeUjzZC3WFFORh82uoFkvJJFtdA5t8ALoWBKZAGwcCbrVjZAoX9i6i35hPjsXBCCA9TANVnlUcgoZD"
    base_url_f = f"https://graph.facebook.com/v17.0/{business_ig_id}?fields=business_discovery.username({user_name})%7Bbiography%2Cname%2Cusername%2Cfollows_count%2Cprofile_picture_url%2Cig_id%2Cfollowers_count%2Cmedia_count%2Cmedia"
    base_url_b = f"%7Bcaption%2Ccomments_count%2Cid%2Cchildren%7Bmedia_url%2Cmedia_type%7D%2Clike_count%2Cmedia_product_type%2Cmedia_type%2Cmedia_url%2Cowner%2Cpermalink%2Ctimestamp%2Cusername%7D%7D&access_token={access_token}"

    data = get_facebook_data((base_url_f+base_url_b))
    if 'business_discovery' in data:
        user_info = extract_user_info(data['business_discovery'])

        #get_media_data
        media_list = []


        for i in range(1):
            media_list.extend(extract_media_info(data['business_discovery']))
            while True:
              after = data.get('business_discovery', {}).get('media', {}).get('paging', {}).get('cursors', {}).get('after', None)
              if not after:
                  break
              next_url = (base_url_f+f".after({after})"+base_url_b)
              data = get_facebook_data(next_url)
              #print(extract_media_info(data['business_discovery']))
              media_list.extend(extract_media_info(data['business_discovery']))
              print(len(media_list))

        save_to_csv([user_info], f"{user_name}.csv")
        save_to_csv(media_list, f"{user_name}_media.csv")
        print("CSV 파일이 성공적으로 생성되었습니다.")
    else:
        print("데이터를 가져오지 못했습니다. URL과 access_token을 확인해주세요.")
